In [6]:
import requests
import yaml
import time
from tqdm import tqdm
from datetime import datetime
import steamreviews
from time import sleep
import os
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute

In [7]:
resp = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2")
app_list = resp.json()
apps = app_list['applist']['apps']

with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)


In [8]:
len(apps)

131982

In [9]:
def review_insert(reviews, appid):
    need_data = ['votes_up', 'votes_funny', 'steam_purchase', 'received_for_free', 'written_during_early_access']
    need_data_dic ={'steam_appid': appid}
        
    for _, review in reviews[0]['reviews'].items():
        author = review['author']
        steamid = author['steamid']
        sql = "SELECT steam_appid FROM review WHERE steam_appid=%s AND steamid=%s" % (appid, steamid)
        if db_function.execute_select(sql):
            return
        
        for k, v in author.items():
            
            if k == 'last_played':
                v = datetime.fromtimestamp(v).strftime("%Y-%m-%d %H:%M:%S")
            need_data_dic[k] = v
        
        for col in need_data:
            need_data_dic[col] = review[col]
        
        placeholders = ', '.join(['%s'] * len(need_data_dic))
        columns = ', '.join(need_data_dic.keys())
        
        sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s )" % ('review', columns, placeholders)
        db_function.execute_insert(sql, list(need_data_dic.values()))


In [ ]:
start_time = time.time()
try_cnt = 0

db_function = None
try:
    db_function = db_execute(db_auth['local_db']["host"], 
                         db_auth['local_db']["port"],
                         db_auth['local_db']["user"],
                         db_auth['local_db']["db"], 
                         db_auth['local_db']["passwd"])
    
    for app in tqdm(apps):
        try_cnt +=1
        current_time = time.time()
        elapsed_time = current_time - start_time

        if try_cnt > 200 and elapsed_time < 300:
            sleep(60)
            try_cnt = 0
            start_time = time.time()
            

        appid=app['appid']
        request_params = dict()
        request_params['language'] = 'korean'

        reviews = steamreviews.download_reviews_for_app_id(appid,chosen_request_params=request_params)
        if reviews[0]['reviews']:
            review_insert(reviews, appid)
        sleep(5)
except:
    raise
finally:
    db_function.curs.close()
    db_function.conn.close()
    

  0%|          | 12/131982 [01:03<192:24:20,  5.25s/it]

[appID = 394430] expected #reviews = 2


  0%|          | 13/131982 [01:08<192:34:34,  5.25s/it]

[appID = 394480] expected #reviews = 1


  0%|          | 14/131982 [01:13<192:33:23,  5.25s/it]

[appID = 394490] expected #reviews = 3


  0%|          | 15/131982 [01:18<192:35:45,  5.25s/it]

[appID = 394510] expected #reviews = 1345
